In [1]:
# add libraries
import requests
import json
import pandas as pd
import os
from pprint import pprint
import time

# Data Acquisition

First have to get PUUID (unique id for players) that is required to search through their matches.

To get PUUID, use LEAGUE-V4 to get a list of players to search up using SUMMONER-V4 to get PUUID. After acquiring PUUID for players, use that PUUID to query in MATCH-V5 to get recent matches. Using those matches, get the timelines for each match at 10 minute mark.

In [11]:
filename = 'api'

In [12]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [13]:
api_key = get_file_contents(filename)

In [14]:
# acquire data from different API endpoints
# League-V4

queue = 'RANKED_SOLO_5x5'
tier = 'DIAMOND'
division = 'I'

# pages of players
# iterate through pages 1-5 to get ~1000 players to use as sample data for now
params = {
    "page": 1
}

url = 'https://na1.api.riotgames.com/lol/'
league_url = f'{url}league/v4/entries/{queue}/{tier}/{division}'

headers = {}
headers["X-Riot-Token"] = api_key

In [15]:
league_data = []
for i in range(5):
    params["page"] = i + 1
    res = requests.get(url = league_url, params = params, headers = headers)
    league_data.append(res.json())

In [16]:
# get the summonerId's to search up
summoner_ids = []
for page in league_data:
    for entry in page:
        summoner_ids.append(entry.get('summonerId'))

In [17]:
len(summoner_ids)

1025

In [18]:
# get the PUUID and save them to a pd dataframe
# work with the first 100 puuids for now
# SUMMONER-V4

summoner_data = []

for id in summoner_ids:
    summoner_url = f'{url}summoner/v4/summoners/{id}'
    res = requests.get(url = summoner_url, headers = headers)
    summoner_data.append(res.json())
    time.sleep(1)

In [19]:
puuids = []
for summoner in summoner_data:
    puuids.append(summoner.get('puuid'))

In [20]:
len(puuids)

1025

In [21]:
puuids[0]

'gCQWB6xb6N_E4zBkzeUJ2dr2b8Kj-Vw67w-tj7S9wrSqwH6cuh_EHIeLLUcPEI2tShxCRRCALOxKgw'

In [22]:
# get the 10 most recent games for puuid
# store into set so no duplicate games are used
last_10_games = []

params = {
    "queue": 420,
    "start": -1,
    "count": 11
}

for puuid in puuids:
    last_10_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    res = requests.get(url = last_10_url, params = params, headers = headers)
    for match_id in res.json():
        last_10_games.append(match_id)
    time.sleep(1)

In [44]:
last_10_games[0]

'NA1_4506172470'

# Feature Engineering

participants 1-5 are blue side

6-10 are red side

WARDS:

WARD_KILL

WARD_PLACED

ITEM_DESTROYED: 2055 (control ward)

KILLS:

KILL_FIRST_BLOOD

CHAMPION_KILLS

OBJECTIVES:

DEATHS:

TURRET_PLATE_DESTROYED

BUILDING_KILL

ELITE_MONSTER_KILL

Each frame is 60 seconds of gameplay.
So for each frame need to record the events by parsing through the request
like wards placed
count total kills from each frame as well

In [71]:
# make a dataframe with matchid as index and features as columns
# first make functions to parse through the frames
# need containers to store the data before making a data frame
# make a class to store the data of a game
class Game:

    def __init__(self, match):
        self.id = ""

        self.winning_team = 0

        self.blue_wards_placed = 0
        self.blue_wards_killed = 0
        self.blue_champions_killed = 0
        self.blue_elites_killed = 0
        self.blue_plates_destroyed = 0
        self.blue_turrets_destroyed = 0
        self.blue_total_gold = 0
        self.blue_average_level = 1
        self.blue_jungle_monsters_killed = 0
        self.blue_minions_killed = 0
        self.blue_first_blood = 0

        self.red_wards_placed = 0
        self.red_wards_killed = 0
        self.red_champions_killed = 0
        self.red_elites_killed = 0
        self.red_plates_destroyed = 0
        self.red_turrets_destroyed = 0
        self.red_total_gold = 0
        self.red_average_level = 1
        self.red_jungle_monsters_killed = 0
        self.red_minions_killed = 0
        self.red_first_blood = 0

        self.get_match_id(match)
        self.set_winning_team(match)
        self.parse_events(match.get('info').get('frames'))

        # some games end before 10 minutes
        try:
            self.get_team_info(match.get('info').get('frames')[10])
        except:
            self.get_team_info(match.get('info').get('frames')[-1])

    def first_blood(self, event):
        if event.get('killType') == 'KILL_FIRST_BLOOD':
            participant = event.get('killerId')
            if participant in range(1,6):
                self.blue_first_blood = 1
            else:
                self.red_first_blood = 1

    def count_wards_placed(self, event):
        if event.get('type') == 'WARD_PLACED':
            participant = event.get('creatorId')
            if participant in range(1,6):
                self.blue_wards_placed += 1
            else:
                self.red_wards_placed += 1

    def count_ward_kills(self, event):
        if event.get('type') == 'WARD_KILL':
            participant = event.get('killerId')
            if participant in range(1,6):
                self.blue_wards_killed += 1
            else:
                self.red_wards_killed += 1

    def count_champion_kills(self, event):
        if event.get('type') == 'CHAMPION_KILL':
            participant = event.get('killerId')
            if participant in range(1,6):
                self.blue_champions_killed += 1
            else:
                self.red_champions_killed += 1

    def count_elite_kills(self, event):
        if event.get('type') == 'ELITE_MONSTER_KILL':
            participant = event.get('killerTeamId')
            if participant == 100:
                self.blue_elites_killed += 1
            else:
                self.red_elites_killed += 1

    def count_turret_plates_destroyed(self, event):
        if event.get('type') == 'TURRET_PLATE_DESTROYED':
            team = event.get('teamId')
            if team == 200:
                self.blue_plates_destroyed += 1
            else:
                self.red_plates_destroyed += 1

    def count_turrets_destroyed(self, event):
        if event.get('type') == 'BUILDING_KILL':
            team = event.get('teamId')
            if team == 200:
                self.blue_turrets_destroyed += 1
            else:
                self.red_turrets_destroyed += 1

    def parse_events(self, frames):
        for frame in frames[0:11]:
            for event in frame.get('events'):
                self.count_wards_placed(event)
                self.count_ward_kills(event)
                self.count_champion_kills(event)
                self.count_elite_kills(event)
                self.count_turret_plates_destroyed(event)
                self.count_turrets_destroyed(event)
                self.first_blood(event)

    def count_team_gold(self, participants):
        for p in range(1,6):
            self.blue_total_gold += participants.get(f'{p}').get('totalGold')
        for p in range(6,11):
            self.red_total_gold += participants.get(f'{p}').get('totalGold')
    
    def team_average_level(self, participants):
        blue_levels = 0
        red_levels = 0
        for p in range(1,6):
            blue_levels += participants.get(f'{p}').get('level')
        for p in range(6,11):
            red_levels += participants.get(f'{p}').get('level')
        self.blue_average_level = blue_levels / 5
        self.red_average_level = red_levels / 5

    def count_team_cs(self, participants):
        for p in range(1,6):
            self.blue_jungle_monsters_killed += participants.get(f'{p}').get('jungleMinionsKilled')
            self.blue_minions_killed += participants.get(f'{p}').get('minionsKilled')
        for p in range(6,11):
            self.red_jungle_monsters_killed += participants.get(f'{p}').get('jungleMinionsKilled')
            self.red_minions_killed += participants.get(f'{p}').get('minionsKilled')
    
    def get_team_info(self, frame):
        participants = frame.get('participantFrames')

        self.count_team_gold(participants)
        self.team_average_level(participants)
        self.count_team_cs(participants)

    def get_match_id(self, match):
        self.id = match.get('metadata').get('matchId')

    def set_winning_team(self, match):
        self.winning_team = match.get('info').get('frames')[-1].get('events')[-1].get('winningTeam')

### Making DataFrame with features

In [46]:
print(len(last_10_games))

10250


In [56]:
last_10_games[1160:1171]

['NA1_4487035412',
 'NA1_4487025995',
 'NA1_4481658693',
 'NA1_4476320211',
 'NA1_4476286115',
 'NA1_4394034652',
 'NA1_4386735463',
 'NA1_4385863580',
 'NA1_4385829631',
 'NA1_4385787432']

In [62]:
# makes a lot of API calls, taking at least 1 second each
games = []
for match_id in last_10_games:
    match_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline"
    res = requests.get(url = match_url, headers = headers)
    if res.status_code == 200:
        match = res.json()
        if match.get('info').get('frameInterval'):
            game = Game(match)
            games.append(game)
            time.sleep(0.8)
    

games_df = pd.DataFrame([vars(g) for g in games])

## Making a new DF of 10000 games
### Has additional features

In [63]:
games_df_large = games_df.drop_duplicates('id')

In [64]:
games_df_large.to_csv('../data/large_sample.csv')

In [70]:
games_df_large['blue_first_blood'].value_counts()

1    4716
0    4610
Name: blue_first_blood, dtype: int64

In [68]:
games_df_large['red_first_blood'].value_counts()

0    9326
Name: red_first_blood, dtype: int64

In [290]:
# games_df.duplicated('id').any()

True

In [291]:
# games_df.shape

(900, 22)

In [292]:
# games_df_small = games_df.drop_duplicates('id')

In [293]:
# games_df_small.shape

(895, 22)

In [294]:
# games_df_small.to_csv('../data/small_sample.csv')